In [2]:
import fxcmpy
TOKEN = "65aa373e2c6d310060a655ac665b46531ecd1adc"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

|ERROR|2022-02-16 21:34:01,280|FXCM reject req trading/update_subscriptions with status 520 and msg <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<meta http-equiv="refresh" content="0">

<title>api-demo.fxcm.com | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" type="text/css" media="screen,projection" />


<script type="text/javascript">
(function(){if(documen

In [3]:
import datetime as dt
from icecream import ic 


start = dt.datetime(2022,2,12)
end = dt.datetime(2022,2,17)
data = con.get_candles('EUR/USD', period='m1', start=start, end=end) #number=250 start=start, end=end
data


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2022-02-13 19:30:00,1.13422,1.13424,1.13424,1.13422,1.13507,1.13505,1.13507,1.13505,2
2022-02-13 19:54:00,1.13434,1.13434,1.13434,1.13434,1.13495,1.13495,1.13495,1.13495,1
2022-02-13 19:58:00,1.13424,1.13424,1.13424,1.13424,1.13505,1.13505,1.13505,1.13505,1
2022-02-13 20:02:00,1.13424,1.13424,1.13424,1.13424,1.13457,1.13455,1.13457,1.13455,2
2022-02-13 20:03:00,1.13425,1.13425,1.13425,1.13425,1.13454,1.13454,1.13454,1.13454,1
...,...,...,...,...,...,...,...,...,...
2022-02-16 16:00:00,1.13648,1.13640,1.13652,1.13637,1.13661,1.13652,1.13664,1.13650,269
2022-02-16 16:01:00,1.13639,1.13640,1.13645,1.13631,1.13651,1.13653,1.13657,1.13643,264
2022-02-16 16:02:00,1.13640,1.13648,1.13652,1.13626,1.13652,1.13660,1.13664,1.13639,243


In [5]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])
# data.to_csv("feb.csv")

In [3]:
from icecream import ic
from datetime import datetime

# d1 = pd.read_csv("dec1.csv")
# d2 = pd.read_csv("dec2.csv")
# d3 = pd.read_csv("dec3.csv")
d3 = pd.read_csv("second_jan.csv")
frames = [d3]
data = pd.concat(frames)
data.index = data.date.values

# # data = data.loc[(data["date"] > datetime.strptime("2022-01-05", "%Y-%m-%d"))]
data = data.drop(["date"], axis=1)
# data = data.iloc[40:3000] # 5831
data
# pd.DataFrame(index=data.date, data=data)

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
2022-01-19 00:00:00,1.13283,1.13289,1.13291,1.13283,1.13295,1.13301,1.13304,1.13295,110
2022-01-19 00:01:00,1.13289,1.13265,1.13289,1.13265,1.13302,1.13278,1.13303,1.13278,103
2022-01-19 00:02:00,1.13265,1.13232,1.13270,1.13226,1.13277,1.13245,1.13283,1.13239,193
2022-01-19 00:03:00,1.13233,1.13242,1.13246,1.13233,1.13245,1.13254,1.13258,1.13245,125
2022-01-19 00:04:00,1.13243,1.13233,1.13243,1.13217,1.13254,1.13246,1.13254,1.13230,160
...,...,...,...,...,...,...,...,...,...
2022-01-27 23:56:00,1.11434,1.11433,1.11434,1.11433,1.11446,1.11446,1.11448,1.11445,7
2022-01-27 23:57:00,1.11433,1.11429,1.11433,1.11429,1.11445,1.11440,1.11446,1.11439,43
2022-01-27 23:58:00,1.11429,1.11430,1.11431,1.11425,1.11439,1.11441,1.11442,1.11435,63
2022-01-27 23:59:00,1.11429,1.11438,1.11439,1.11429,1.11440,1.11450,1.11451,1.11439,48


In [6]:
# import ta
# from ta.trend import *
# from ta.momentum import *
from IPython.display import display


# data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
# data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# # Stochastic need to fix
# data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)

# data

In [4]:
import importlib
import strategy
importlib.reload(strategy)
from strategy import BaseStrategy
import pandas as pd
import numpy as np
from ta.trend import *
from ta.momentum import *

class SMAStrategy(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "SELL"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "BUY"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        
        df.loc[(df["sma_signal"] == "BUY")  & (df["stoch"] >= 60) & (df["stoch_signal"] >= 60), "stoch_signal_f"] = "CHUTIYA"
        df.loc[(df["sma_signal"] == "SELL")  & (df["stoch"] <= 40) & (df["stoch_signal"] <= 40), "stoch_signal_f"] = "CHUTIYA" 
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return self.signal_found(df.loc[(df["stoch_signal_f"]== "CHUTIYA")].tail(1))
        
    
    def apply_indicator(self):
        super(SMAStrategy, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)


    
# strategy = SMAStrategy()
# # for i in range(51,100):
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# df = strategy.get_signal()
# df = df.loc[(df["SMA_SIGNAL"]=="BUY") | (df["SMA_SIGNAL"]=="SELL")]


# df = df.loc[(df["STOCH_SIGNAL_F"] == "CHUTIYA" )]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["SMALL_SMA"] <= df.bidopen) & (df["SMALL_SMA"] <= df.bidclose) &
#            (df["LARGE_SMA"] <= df.bidopen) & (df["LARGE_SMA"] <= df.bidclose)) | 
#            ((df["SMA_SIGNAL"] == "SELL") & (df["SMALL_SMA"] >= df.bidopen) & (df["SMALL_SMA"] >= df.bidclose) &
#            (df["LARGE_SMA"] >= df.bidopen) & (df["LARGE_SMA"] >= df.bidclose))
#            ]
# df = df.loc[((df["SMA_SIGNAL"] == "BUY") & (df["RSI"] >= 70)) | ((df["SMA_SIGNAL"] == "SELL") & (df["RSI"] <= 30))]
# df = df.loc[:, ["bidopen","bidclose","SMA_SIGNAL", "SMALL_SMA", "LARGE_SMA", "RSI", "STOCH","STOCH_SIGNAL"]]
# df.drop(["askopen","askclose", "askhigh", "tickqty"], axis=1)
# df
# df.insert(loc = 0,column = 'Strategy Valid?',value = '<input type="checkbox" />')
# df.tail(50)
# df.style
# df

In [1014]:
class SMAStrategy1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "DOWN_CROSSOVER"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "UP_CROSSOVER"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        # df["entry_signal"] = np.select([(df["sma_signal"] == "UP_CROSSOVER"),(df["sma_signal"] == "DOWN_CROSSOVER")],["BUY","SELL"]) 
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        new_df = df[last_sma_crossover.name:df.iloc[-1].name]
        new_df = new_df.iloc[1: , :]
        signal = None
        data = None
        amt = 0.00003
        if last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            candle_counter = 0
            iter_counter = 0
            signal = "SELL"
            for row in new_df.itertuples():
                if row.max_bid+amt >= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
                # ic(row)
            # sma_touched_df = new_df.head(2).loc[(new_df["max_bid"] >= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "SELL"
            # data = new_df.head(2).tail(1)
        elif last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            candle_counter = 0
            iter_counter = 0
            for row in new_df.itertuples():
                if row.min_bid-amt <= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
            
            # sma_touched_df = new_df.head(2).loc[(new_df["min_bid"] <= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "BUY"
            # data = new_df.head(2).tail(1)
        if signal and not data.empty:
            # ic(signal)
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
        #     # ic(d)
            return d
        # ic(last_sma_crossover)
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        

# strategy = SMAStrategy1()
# strategy.dataframe = data #.iloc[120:187]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).tail(50)
# dd

In [17]:
class SMAStrategy2(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER" and \
                    last_sma_crossover.min_bid > last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and \
                    last_sma_crossover.max_bid < last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).iloc[1650:1700]

In [5]:
class SMAStrategy3Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        signal = self.entry_signal
        target = self.__class__.get_target_price(signal.entry_price, self.target, signal.entry_signal)
        stoploss = self.__class__.get_stoploss_price(signal.entry_price, self.stoploss, signal.entry_signal)
        df_last_date = df.tail(1).index[0]
        temp_df = df[:signal.Index][-10:]
        rows_min = min(temp_df.min_bid)
        rows_max = max(temp_df.max_bid)
        slp_price= 0.0002
        tgp_price = 0.0002
        new_df = df[signal.Index:df_last_date].iloc[1: , :][:100]
        exit_price = 0
        exit_time = None
        status = ""
        if signal.entry_signal == "BUY":
            last_row = None
            new_sl = min(signal.bidopen - slp_price, rows_min)
            new_tgp_row = None
            for row in new_df.itertuples():
                if last_row == None:
                    last_row = row
                if new_tgp_row == None:
                    if row.max_bid >= signal.bidclose + tgp_price:
                        new_tgp_row = row
                # if new_tgp_row and (row.min_bid <= signal.bidclose):
                #     exit_price = row.min_bid
                #     exit_time = row.Index
                #     status = "SLPR"
                #     break   
                elif row.max_bid >= target:
                    if row.rsi < 67:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.min_bid <= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.min_bid <= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                elif last_row.rsi > 70 and row.rsi < 70:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                # elif row.rsi < 30:
                #     if last_row.rsi > 30 and row.rsi < 30:
                #         if (pd.Timestamp(row.Index) - pd.Timestamp(signal.Index)) / pd.Timedelta(minutes=1) <= 15:
                #             exit_price = row.bidclose
                #             exit_time = row.Index
                #             status = "SLP"
                #             break
                #     elif row.rsi < 27:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "SLP"
                #         break
                last_row = row
        elif signal.entry_signal == "SELL":
            last_row = None
            new_sl = max(rows_max, signal.bidopen + slp_price)
            new_tgp_row = None
            for row in new_df.itertuples():
                if last_row == None:
                    last_row = row
                if new_tgp_row == None:
                    if row.min_bid <= signal.bidclose - tgp_price:
                        new_tgp_row = row
                # if new_tgp_row and (row.max_bid >= signal.bidclose):
                #     exit_price = row.max_bid
                #     exit_time = row.Index
                #     status = "SLPR"
                #     break  
                elif row.min_bid <= target:
                    if row.rsi > 33:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.max_bid >= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.max_bid >= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                elif last_row.rsi < 30 and row.rsi > 30:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                # elif row.rsi > 70:
                #     if last_row.rsi < 70 and row.rsi > 70:
                #         if (pd.Timestamp(row.Index) - pd.Timestamp(signal.Index)) / pd.Timedelta(minutes=1) <= 15:
                #             exit_price = row.bidclose
                #             exit_time = row.Index
                #             status = "SLP"
                #             break
                #     elif row.rsi > 73:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "SLP"
                #         break
                last_row = row
        # print(exit_price, exit_time, status)
        # return new_df
        if all([exit_price, status, exit_time]):
            return exit_price, exit_time, status
        return 0,0,0

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value
# print(bc)
# tragedy = SMAStrategy3Exit(target=0.0008, stoploss=0.0003)
# tragedy.dataframe = data
# tragedy.entry_signal = test_row
# tragedy.is_valid_dataframe()
# dd= tragedy.get_signal()
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).head(50) .iloc[1:60]
# dd

In [92]:
class SMAStrategy3(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    
    def signal_found(self, data):
        return data.to_dict()
    
    def get_last_signal(self):
        df = self.dataframe.copy()
        signal_found = False
        
        num = list(range(-2,-8, -1))
        for i in range(1, len(num)):
            second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[num[i-1]]
            last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[num[i]]
            new_df = df[last_sma_crossover.name:second_last_sma_crossover.name]
            new_df = new_df.iloc[::-1]
            signal = None
            data = pd.DataFrame()


            if second_last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
                signal = "BUY"
                valid_row = 0
                for row in new_df[1:4].itertuples():
                    if max(row.bidopen, row.bidclose) < row.small_sma:
                        valid_row += 1
                if valid_row >= 2:
                    data = new_df[second_last_sma_crossover.name : second_last_sma_crossover.name]
            elif second_last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
                signal = "SELL"
                valid_row = 0
                for row in new_df[1:4].itertuples():
                    if min(row.bidopen, row.bidclose) > row.small_sma:
                        valid_row += 1
                if valid_row >= 2:
                    data = new_df[second_last_sma_crossover.name : second_last_sma_crossover.name]

            if signal and not data.empty:
                d_date = data.index[0]
                d = data.to_dict()
                d["entry_price"] = d["bidclose"]
                d["entry_signal"] = {d_date: signal}
                return d
        return None
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        last_signal = next(pd.DataFrame(self.get_last_signal()).itertuples())
        ic(last_signal)
        # exit_of_last_signal = 
        # return last_signal
        # ic(self.last_signal)
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

strategy = SMAStrategy3()
strategy.dataframe = data[:300]
# print(data[:298].tail(1))

strategy.is_valid_dataframe()
dd = strategy.get_signal()
dd


# tragedy = SMAStrategy3Exit(target=0.0008, stoploss=0.0003)
# tragedy.dataframe = data
# tragedy.entry_signal = dd
# tragedy.is_valid_dataframe()
# ddd= tragedy.get_signal()
# ddd
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty","bidhigh", "bidlow", "asklow",
#         ], axis=1).head(50)
# data.iloc[3000:4000].tail(15

ic| last_signal: Pandas(Index=Timestamp('2022-02-14 01:45:00'), bidopen=1.13524, bidclose=1.13529, bidhigh=1.13531, bidlow=1.13522, askopen=1.13536, askclose=1.13541, askhigh=1.13544, asklow=1.13534, tickqty=137, max_bid=1.13531, min_bid=1.13522, small_sma=1.1353342857142859, large_sma=1.13544, rsi=33.751700474006014, rsi_signal='UP_REVERSE', sma_signal='', entry_price=1.13529, entry_signal='BUY')


{'bidopen': {Timestamp('2022-02-14 02:06:00'): 1.1349},
 'bidclose': {Timestamp('2022-02-14 02:06:00'): 1.13492},
 'bidhigh': {Timestamp('2022-02-14 02:06:00'): 1.13492},
 'bidlow': {Timestamp('2022-02-14 02:06:00'): 1.13483},
 'askopen': {Timestamp('2022-02-14 02:06:00'): 1.13502},
 'askclose': {Timestamp('2022-02-14 02:06:00'): 1.13504},
 'askhigh': {Timestamp('2022-02-14 02:06:00'): 1.13505},
 'asklow': {Timestamp('2022-02-14 02:06:00'): 1.13495},
 'tickqty': {Timestamp('2022-02-14 02:06:00'): 101},
 'max_bid': {Timestamp('2022-02-14 02:06:00'): 1.13492},
 'min_bid': {Timestamp('2022-02-14 02:06:00'): 1.13483},
 'small_sma': {Timestamp('2022-02-14 02:06:00'): 1.1349228571428571},
 'large_sma': {Timestamp('2022-02-14 02:06:00'): 1.135044},
 'rsi': {Timestamp('2022-02-14 02:06:00'): 31.736606078736898},
 'rsi_signal': {Timestamp('2022-02-14 02:06:00'): 'UP_REVERSE'},
 'sma_signal': {Timestamp('2022-02-14 02:06:00'): ''},
 'entry_price': {Timestamp('2022-02-14 02:06:00'): 1.13492},
 'e

In [178]:
class SMAStrategy2Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to7754w_dict()
    6665qwerr
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        # if signal and not data.empty:
        #     d_date = data.index[0]
        #     d = data.to_dict()
        #     d["entry_price"] = d["bidclose"]
        #     d["entry_signal"] = {d_date: signal}
        #     return d
        return df
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1)
# dd

In [7]:
test_data = data.copy() #[5000:10000]
import backtest
import strategy
importlib.reload(strategy)
importlib.reload(backtest)
from backtest import BackTestStrategy
importlib.reload(strategy)
importlib.reload(backtest)


sma_backtest = BackTestStrategy(SMAStrategy3, test_data, exit_strategy=SMAStrategy3Exit, target=0.0008,stoploss=0.0005, order_quantity=30000)
sma_backtest.start_backtesting()

'backtesting has been completed, please generate report'

In [12]:
import strategy
importlib.reload(strategy)
import backtest
importlib.reload(backtest)
kd = sma_backtest.get_report()
# sma_backtest.signal_data
# kd["max_bid"] = kd[["bidopen","bidclose","bidlow","bidhigh"]].max(axis=1)2022-01-03 10:53:00
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
print(len(list(filter(lambda k : k == "TGP", kd.status))) , "Total TGP")
print(len(list(filter(lambda k : k == "SLP", kd.status))) , "Total SLP")
print(len(list(filter(lambda k : k == "SLPR", kd.status))) , "Total SLPR")
print(len(list(filter(lambda k : k in ["SL", "SLP"], kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k in ["TG","TGP"], kd.status))) , "Total Target")
print(f"Total Amount Earned {kd.pl.sum()}")
print(f"From Date: {kd.iloc[1].name}")
print(f"To Date: {kd.iloc[-1].name}")
kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
        "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
ic(kd[kd["status"] == "SLP"].pl.sum())
ic(kd[kd["status"] == "SLPR"].pl.sum())
ic(kd[kd["status"] == "TGP"].pl.sum())
ic(kd[kd["status"] == "SL"].pl.sum())
ic(kd[kd["status"] == "TG"].pl.sum())
len(kd[kd["status"] == 0].pl)


ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
ic| kd[kd["status"] == "SLPR"].pl.sum(): -389.6999999999817
ic| kd[kd["status"] == "TGP"].pl.sum(): 491.6999999999903

3 Total Stoploss
35 Total Target
54 Total TGP
0 Total SLP
36 Total SLPR
3 Total Stoploss
89 Total Target
Total Amount Earned 892.4999999999852
From Date: 2022-02-13 22:26:00
To Date: 2022-02-16 15:50:00
(151, 11)



ic| kd[kd["status"] == "SL"].pl.sum(): -57.90000000000628
ic| kd[kd["status"] == "TG"].pl.sum(): 848.3999999999826


23

In [17]:
kd.to_csv("modified_strategy1.csv")


In [43]:
ss = pd.DataFrame(sma_backtest.signal_data)
for test_row in ss.itertuples():
    if pd.Timestamp(test_row.Index) == pd.Timestamp("2022-01-21 11:12:00"):
        break

In [57]:
data.tail(1)

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2022-02-03 15:07:00,1.13858,1.13889,1.13893,1.13858,1.13872,1.13901,1.13905,1.1387,374


packet queue is empty, aborting
|ERROR|2022-02-11 10:01:27,781|packet queue is empty, aborting
packet queue is empty, aborting
|ERROR|2022-02-11 17:18:10,246|packet queue is empty, aborting
packet queue is empty, aborting
|ERROR|2022-02-12 20:36:10,762|packet queue is empty, aborting


In [618]:
# ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
# ic| kd[kd["status"] == "SLPR"].pl.sum(): -572.6999999999948
# ic| kd[kd["status"] == "TGP"].pl.sum(): 417.3000000000159
# ic| kd[kd["status"] == "SL"].pl.sum(): -52.80000000000618
# ic
# 2 Total Stoploss
# 19 Total Target
# 46 Total TGP
# 0 Total SLP
# 56 Total SLPR
# 2 Total Stoploss
# 65 Total Target
# Total Amount Earned 247.5000000000027
# From Date: 2022-01-24 13:19:00
# To Date: 2022-01-27 23:58:00
# (137, 12)
# | kd[kd["status"] == "TG"].pl.sum(): 455.6999999999877

# ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
# ic| kd[kd["status"] == "SLPR"].pl.sum(): -572.6999999999948
# ic| kd[kd["status"] == "TGP"].pl.sum(): 417.3000000000159
# ic| kd[kd["status"] == "SL"].pl.sum(
# 2 Total Stoploss
# 19 Total Target
# 46 Total TGP
# 0 Total SLP
# 56 Total SLPR
# 2 Total Stoploss
# 65 Total Target
# Total Amount Earned 247.5000000000027
# From Date: 2022-01-24 13:19:00
# To Date: 2022-01-27 23:58:00
# (137, 12)
# ): -52.80000000000618
# ic| kd[kd["status"] == "TG"].pl.sum(): 455.6999999999877

In [49]:
next(pd.DataFrame(dk).itertuples())

Pandas(Index=Timestamp('2022-01-31 05:06:00'), bidopen=1.11604, bidclose=1.11606, bidhigh=1.11606, bidlow=1.11604, askopen=1.11617, askclose=1.11618, askhigh=1.11619, asklow=1.11617, tickqty=20, max_bid=1.11606, min_bid=1.11604, small_sma=1.1160442857142858, large_sma=1.1160679999999998, rsi=37.12628874801934, rsi_signal='UP_REVERSE', sma_signal='', entry_price=1.11606, entry_signal='BUY')

In [88]:
num = list(range(-3,-8, -1))
for i in range(1, len(num)):
    print(num[i-1])
    print(num[i])
    print("break")

-3
-4
break
-4
-5
break
-5
-6
break
-6
-7
break
